# Agent Gateway Quickstart

Agent Gateway is a multi-agent framework that offers native support for Snowflake tools. 

The system can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Agent Gateway does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing the agent.

# Agent Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [ ]:
import os
#os.environ["LOGGING_LEVEL"] = "DEBUG"

In [1]:
from agent_gateway import Agent
from agent_gateway.tools import CortexSearchTool, CortexAnalystTool, PythonTool
from snowflake.snowpark import Session
import os
import streamlit as st


from snowflake.snowpark.context import get_active_session

snowpark = get_active_session()

In [ ]:
from agent_gateway.tools.utils import generate_demo_services

generate_demo_services(snowpark)


## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [ ]:
use database CUBE_TESTING;
use schema PUBLIC;

In [2]:

search_config = {
    "service_name": "CUBE_TESTING.PUBLIC.SEC_SEARCH_SERVICE",
    "service_topic": "Snowflake's business,product offerings,and performance.",
    "data_description": "Snowflake annual reports",
    "retrieval_columns": ["CHUNK", "RELATIVE_PATH"],
    "snowflake_connection": snowpark,
    "k": 10,
}

analyst_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "CUBE_TESTING.PUBLIC.ANALYST",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowflake_connection": snowpark,
}

## Python Tool Configuration

Configuring a Python Tool for the Agent Gateway requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [3]:
import requests


def html_crawl(url):
    response = requests.get(url)
    return response.text


python_crawler_config = {
    "tool_description": "reads the html from a given URL or website",
    "output_description": "html of a webpage",
    "python_func": html_crawl,
}

## Agent Config

After the tools have been configured, initialize them and configure the agent.

In [4]:
annual_reports = CortexSearchTool(**search_config)
sp500 = CortexAnalystTool(**analyst_config)
web_crawler = PythonTool(**python_crawler_config)

snowflake_tools = [annual_reports, sp500, web_crawler]
agent = Agent(snowflake_connection=snowpark, tools=snowflake_tools, max_retries=3)

2025-03-13 17:24:15,091 - AgentGatewayLogger - INFO - Cortex Search Tool successfully initialized
2025-03-13 17:24:15,092 - AgentGatewayLogger - INFO - Cortex Analyst Tool successfully initialized
2025-03-13 17:24:15,093 - AgentGatewayLogger - INFO - Python Tool successfully initialized
2025-03-13 17:24:15,094 - AgentGatewayLogger - INFO - Cortex gateway successfully initialized


# Agent Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with the Agent Gateway. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

## Structured Data Questions

In [5]:
response = agent("What is the market cap of Apple?")

2025-03-13 17:24:22,280 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


{'output': 'The market cap of Apple is $3,019,131,060,224 or approximately $3.02 trillion.',
 'sources': [{'tool_type': 'cortex_analyst',
   'tool_name': 'sp500_semantic_model_cortexanalyst',
   'metadata': [{'Table': 'cube_testing.public.sp500'}]}]}

In [ ]:
st.text(response)

In [6]:
agent("Which company has the bigger EBITDA, Apple or MSFT?")

2025-03-13 17:24:33,011 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-13 17:24:33,012 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


{'output': "Apple has a bigger EBITDA of $129,629,003,776 compared to Microsoft's $125,981,999,104.",
 'sources': [{'tool_type': 'cortex_analyst',
   'tool_name': 'sp500_semantic_model_cortexanalyst',
   'metadata': [{'Table': 'cube_testing.public.sp500'}]}]}

# Unstructured Data Questions

In [7]:
agent("How many customers does Snowflake have?")

2025-03-13 17:24:49,879 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task


{'output': 'As of January 31, 2024, Snowflake had 9,437 total customers.',
 'sources': [{'tool_type': 'cortex_search',
   'tool_name': 'sec_search_service_cortexsearch',
   'metadata': [{'RELATIVE_PATH': '2024_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2023_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2022_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2021_10k_snowflake.pdf'}]}]}

In [8]:
agent(
    "On which platforms can I host Snowflake according to this documentation page https://docs.snowflake.com/en/user-guide/intro-cloud-platforms"
)

2025-03-13 17:24:57,337 - AgentGatewayLogger - INFO - running html_crawl task


{'output': 'Snowflake can be hosted on the following cloud platforms: Amazon Web Services (AWS), Google Cloud Platform (GCP), and Microsoft Azure (Azure).',
 'sources': [{'tool_type': 'custom_tool',
   'tool_name': 'html_crawl',
   'metadata': [None]}]}

## Unstructured + Structured Data Questions

In [ ]:
agent(
    "What is the market cap of each of the cloud providers mentioned in Snowflake's annual report?"
)

In [9]:
resposne = agent(
    "What is the market cap of each of the cloud providers mentioned in Snowflake's annual report?"
)

2025-03-13 17:25:14,329 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task
2025-03-13 17:25:15,815 - AgentGatewayLogger - INFO - running summarize task
2025-03-13 17:25:17,261 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-13 17:25:21,736 - AgentGatewayLogger - ERROR - Your request is unclear. Consider rephrasing your request to one of the following suggestions: ['What is the market cap of Amazon.com, Inc., Microsoft Corporation, and Alphabet Inc.?', 'What is the market cap of the companies that own Amazon Web Services (AWS), Microsoft Azure, and Google Cloud Platform (GCP)?']
2025-03-13 17:25:30,092 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-13 17:25:33,332 - AgentGatewayLogger - ERROR - Unable to generate a valid SQL Query. The question is asking for the market cap of companies that own Amazon Web Services (AWS), but the semantic data model does not provide information about the 

{'output': 'The market cap of Amazon.com, Inc. is $1,917,936,336,896, the market cap of Microsoft Corporation is $3,150,184,448,000, and the market cap of Alphabet Inc. is $2,164,350,779,392.',
 'sources': [{'tool_type': 'cortex_search',
   'tool_name': 'sec_search_service_cortexsearch',
   'metadata': [{'RELATIVE_PATH': '2023_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2024_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2022_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2021_10k_snowflake.pdf'}]},
  {'tool_type': 'cortex_analyst',
   'tool_name': 'sp500_semantic_model_cortexanalyst',
   'metadata': [{'Table': 'cube_testing.public.sp500'}]}]}

In [ ]:
import streamlit as st

st.markdown(resposne)